# Retrieve and download the raw Youtube video

In [9]:
from pytubefix import YouTube

In [14]:
video_link = "https://www.youtube.com/watch?v=45KmZUc0CzA"
data_folder = "../data/raw"


yt_item = YouTube(video_link)
print(yt_item.title)

stream = yt_item.streams.get_highest_resolution()
stream.download(output_path=data_folder, filename=f"{yt_item.title}.mp4")

Janja Garnbret Goes Gold at Paris 2024 🥇 👑 | Full Replay all climbs


'/home/alex/Code/hightlight-bouldering/notebooks/../data/raw/Janja Garnbret Goes Gold at Paris 2024 🥇 👑 | Full Replay all climbs.mp4'

# Capture highlight moments from the video

## Naive and personal annotation
My first instinct is to try and select moments from the video that I personally consider as highlight moments, and then try to figure out which aspects from the video can be used to automate the process.

Here follows the video highlight moments timestamps:

Janja Garnbret Goes Gold at Paris 2024 🥇 👑 | Full Replay all climbs
    - 0:59 - 1:07
    - 1:26 - 1:35
    - 2:30 - 2:45
    - 2:52 - 3:00
    - 4:42 - 5:17
    - 7:04 - 7:34

## Observations 

For this particular video, the video edits could be a great clue, as the olympic rings appear during transitions before and after a replay. There is also a specific sound at the beginning of the replay, when the olympic rings appear on the screen. It is important to note that this solution would not generalize well with any other video.

A more generalizable solution would be to detect when a single person is being the subject of the frame; we would expect these frames to "matter more" and, if there are enough consecutive frames, it would constitute a highlight moment.

Another general solution related to bouldering is the use of pose estimation, and to find a way to detect rapid changes in poses. This idea would enhance the person detection idea. The issue is how to determine what a "rapid change in poses" actually means.

Crowd noise could be an indicator of the relevance of a moment during the video, but this is hard to isolate from the commentators, and I feel that crowd noise was tuned down in the video.

---

Before going toward coding, I will rate the ideas on specific criterions to help me decide which solution to go to.

| Idea | Ease to Develop | Generalization to Other Videos | Expected Relevance |
|------|------------------|-------------------------------|--------------------|
| Olympic Rings and Sound | +++ | + | +++ |
| Single Person Detection | ++ | +++ | ++ |
| Pose Estimation and Rapid Changes | + | +++ | +++ |
| Crowd Noise Analysis | + | + | ++ |
